In [41]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, TensorDataset, Subset
import torch.nn.functional as F
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import plotly.graph_objects as go
import cufflinks as cf
from sklearn.preprocessing import LabelEncoder
cf.go_offline(connected = True)


from mrcnn.visualize import display_instances
from mrcnn import visualize

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
import matplotlib.pyplot as plt
from mrcnn.visualize import * 

from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import numpy as np
import matplotlib.pyplot as plt

import os
import random
import datetime
import re
import math
import logging
from collections import OrderedDict
import multiprocessing
import numpy as np
import tensorflow.compat.v1 as tf
import tensorflow as tf
import keras
import keras.backend as K
import keras.layers as KL
import keras.engine as KE
import keras.models as KM
import keras.callbacks

from PIL import Image, ImageDraw
from PIL import Image
import skimage 
import pandas as pd
import numpy as np

from mrcnn import utils

tf.compat.v1.disable_eager_execution()

In [2]:
class Net(nn.Module):
    def __init__(self,input_dim,output_dim):
        super().__init__()
        
        self.linear_layers = nn.Sequential(nn.Linear(input_dim, 200),
                                          nn.LeakyReLU(0.1),
                                          nn.Linear(200,100),
                                          nn.LeakyReLU(0.1),
                                          nn.Linear(100,50),
                                          nn.LeakyReLU(0.1),
                                          nn.Linear(50,20),
                                          nn.LeakyReLU(0.1),
                                          nn.Linear(20,5),
                                          nn.LeakyReLU(0.1),
                                          nn.Linear(5, output_dim), 
                                           # 최종 결과는 (20, 3) 이 되므로, dim=-1 로 label 확률값이 들어 있는 마지막 차원을 지정해줘야 함
                                          nn.LogSoftmax(dim = -1)
                                          )
        
    def forward(self,x):
        y = self.linear_layers(x)
        
        return y

In [3]:
model_1 = torch.load('model_1.pt')

In [4]:
class TestConfig(Config):
    NAME = "test"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 13 + 1
    BATCH_SIZE = 1
    
model = MaskRCNN(mode = 'inference',config = TestConfig(), model_dir= '')

model.load_weights('mask_rcnn_deepfashion2_0100.h5', by_name= True)

Instructions for updating:
box_ind is deprecated, use box_indices instead
Instructions for updating:
Use fn_output_signature instead
Instructions for updating:
Use `tf.cast` instead.


In [5]:
import os

def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")


In [11]:
import urllib.request
import tqdm

def make_image_bouding_file(name, osdir):
    
    lst_small = []
    img_dir = osdir+'/' + name
    

    img_0 = Image.open(img_dir)
    img_1 = skimage.io.imread(img_dir)
    
    try:
        A = model.detect([img_1])    
    except:
        class_name = 'Not_Detected'
        lst_small.append([name,class_name,0,0,[None,None,None,None]])
        print('오류 발생')
        return lst_small


    
    # 일단 그림은 탐지되는 경우에만 Applied 폴더에 넣기로 함!
    if len(A[0]['rois']) == 0:
        print('인식되지 않습니다.')
        
        class_name = 'Not_Detected'

        
        lst_small.append([name,class_name,0,0,[None,None,None,None]])
        pass
    
    else:       
        
        y1, x1, y2, x2= A[0]['rois'][0]
        
        if len(A[0]['rois']) != 1:
            for num in range(len(A[0]['rois'])):
                image = img_0.copy()
                
                y1, x1, y2, x2= A[0]['rois'][num]
                class_name = class_lst[A[0]['class_ids'][num]] 
                score = A[0]['scores'][num]
    
                lst_small.append([str(name)+'_'+str(num) ,class_name,score,1,[x1,y1,x2,y2]])

                #### 그림 그리는 부분 #########
                draw = ImageDraw.Draw(image)
                # Draw the bounding box with red lines
                draw.rectangle((x1,y1,x2,y2),
                       outline=(255, 0, 0), # Red in RGB
                       width=3)             # Line width
                
                ### 그림 저장 ###
                image = pil_draw_rect_with_label(image, (x1, y1), (x2, y2),class_name)
                image.save(osdir+'/' + str(name).split('.jpg')[0]+'_'+str(num) + '_converted.jpg')

        else:
            image = img_0.copy()
            y1, x1, y2, x2= A[0]['rois'][0]
            class_name = class_lst[A[0]['class_ids'][0]]
            score = A[0]['scores'][0]
            
            lst_small.append([str(name) ,class_name,score,1,[x1,y1,x2,y2]])

            #### 그림 그리는 부분 #########
            draw = ImageDraw.Draw(image)
            # Draw the bounding box with red lines
            draw.rectangle((x1,y1,x2,y2),
                   outline=(255, 0, 0), # Red in RGB
                   width=3)             # Line width

            ### 그림 저장 ###
            image = pil_draw_rect_with_label(image, (x1, y1), (x2, y2),class_name)
            image.save(osdir+'/' + str(name).split('.jpg')[0] + '_converted.jpg')            
            
                
    return lst_small
    

    

In [12]:
from PIL import Image
import tqdm

In [13]:
class_lst =[
    "short sleeve top",
    "long sleeve top",
    "short sleeve outwear",
    "long sleeve outwear",
    "vest",
    "sling",
    "shorts",
    "trousers",
    "skirt",
    "short sleeve dress",
    "long sleeve dress",
    "vest dress",
    "sling dress",
    'skirt_2'
  ]

In [14]:
def pil_draw_rect_with_label(image, point1, point2, text, font_color=(255, 0, 0)):
    draw = ImageDraw.Draw(image)
    draw.rectangle((point1, point2), outline=(0, 0, 255), width=3)

    x1, y1 = point1
    imageFont = ImageFont.load_default()
    text_width, text_height = imageFont.getsize(text) 
    draw.rectangle(((x1, y1 + text_height), (x1 + text_width, y1 )), fill=(0, 0, 255)) #채워진 사각형
#     draw.text((x1, y1 - text_height), text, font=imageFont, fill=font_color)
#     draw.text((x1, y1 ), text, font=imageFont, fill=font_color)

    return image

In [15]:
os.listdir('Total_baseline_data/인플루언서정보_샘플2')[2:]

['@jelly_wony',
 '@jess.02.23',
 '@jin.wonder',
 '@jindalorian',
 '@jin_pyo_is',
 '@jung_staas',
 '@kimchiz_man',
 '@kimyannnnngh',
 '@kj_m.w',
 '@kxyxn',
 '@k_8_8bsoo',
 '@lamode.seoul',
 '@lil_0uzi_vert',
 '@loolinmx',
 '@lxx.s.y_',
 '@malko_bee',
 '@mavlfit',
 '@mini.d31',
 '@moodonx2',
 '@my_own_way_____',
 '@m_n__day',
 '@odor_bubu',
 '@oneh6_',
 '@one_r_k',
 '@oohsehun',
 '@oytoyt_',
 '@ro.seon',
 '@rozley._.y',
 '@rupinydaily',
 '@ruri.kim',
 '@s.s_jun',
 '@shura_twins_korea',
 '@siaestival',
 '@skuukzky',
 '@slglf',
 '@so_h_appy',
 '@so_j2',
 '@so_love_so_',
 '@sualboys',
 '@sympa_young',
 '@s_uz_.zzy__2',
 '@tlrou.vee',
 '@tt__yl',
 '@uuuuk_2_',
 '@velyjuu',
 '@win.on_',
 '@wnsgurrha',
 '@xixxeeonx_4',
 '@y._.dulcet',
 '@yangkoon__dl',
 '@yeenstyle_',
 '@yj_mark',
 '@ys971003__',
 '@yulkeem___',
 '@yummy.__.do',
 '@y_mood_h',
 '@zxcvr0626',
 '@z___meme']

In [35]:
'Total_baseline_data/인플루언서정보_샘플2/' + files + '/images' + '/' + file.split('.webp')[0]

'Total_baseline_data/인플루언서정보_샘플2/@z___meme/images/3007463791760602227.jpg'

In [38]:
from PIL import Image

lst_score_total = []
for files in tqdm.tqdm(os.listdir('Total_baseline_data/인플루언서정보_샘플2')):

    lst_big = []
    num = 0
    for file in os.listdir('Total_baseline_data/인플루언서정보_샘플2/' + files + '/images' ):
        
        if '.webp' in file:
            im = Image.open('Total_baseline_data/인플루언서정보_샘플2/' + files + '/images' + '/' + file).convert('RGB')
            im.save('Total_baseline_data/인플루언서정보_샘플2/' + files + '/images' + '/' + file.split('.webp')[0] + '.jpg','jpeg')

100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [00:01<00:00, 44.17it/s]


In [39]:
lst_score_total = []
for files in tqdm.tqdm(os.listdir('Total_baseline_data/인플루언서정보_샘플2')[43:]):

    lst_big = []
    num = 0
    for file in tqdm.tqdm(os.listdir('Total_baseline_data/인플루언서정보_샘플2/' + files + '/images' )):
        
        print(file)
        lst_small = []
        if '.jpg' in file:
            lst_small.append(make_image_bouding_file(file , 'Total_baseline_data/인플루언서정보_샘플2/' + files + '/images'))
            
            if len(lst_small) != 0 :
                for small in lst_small:
                    lst_big.append(small)
                    
            else:
                lst_big.append(lst_small[0])
                    
    lst_big_2 = []
    for i in lst_big:
        if len(i) == 1:
            lst_big_2.append(i[0])

        else:
            for j in i:
                lst_big_2.append(j)
    
             
        
    df_lst_small = pd.DataFrame(lst_big_2)
    df_lst_small.columns = ['file_name','class_name','score','deteced','coordinate']
    
    df_lst_small.to_csv('Total_baseline_data/인플루언서정보_샘플2/' + files +'/' + files +'.csv')
    
    lst_score_total.append(lst_big)
    


    

  0%|                                                                                          | 0/120 [00:00<?, ?it/s]

2911815029923589796
2911815029923589796.jpg


C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning:

getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.


  2%|█▎                                                                                | 2/120 [00:02<02:21,  1.20s/it]

2911815029923589796.webp
2911815029923589796jpg
2915442350475811374
2915442350475811374.jpg



  5%|████                                                                              | 6/120 [00:04<01:23,  1.36it/s]

...


 93%|████████████████████████████████████████████████████████████████████████████▏     | 52/56 [01:03<00:04,  1.25s/it]

3006828589023587941
3006828589023587941.jpg



 96%|███████████████████████████████████████████████████████████████████████████████   | 54/56 [01:06<00:02,  1.24s/it]

3007463791760602227
3007463791760602227.jpg



100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [20:12<00:00, 71.30s/it]


In [27]:
lst_score_total = []
num = 0
for idx,files in enumerate(os.listdir('Total_baseline_data/인플루언서정보_샘플2')):
    if files == '@tlrou.vee':
        print(idx,files)
    
    num += 1

43 @tlrou.vee


In [24]:
lst_big_2 = []
for i in lst_big:
    if len(i) == 1:
        lst_big_2.append(i[0])
        
    else:
        for j in i:
            lst_big_2.append(j)
    


In [18]:
lst_score_total

[[[['3007628629936416873.jpg',
    'Not_Detected',
    0,
    0,
    [None, None, None, None]]]]]

In [ ]:
for i in lst_score_total[0]:
    print(i[0])
[i for i in ]